# Azure OpenAI Service Embeddings 基礎
Embeddings とは、機械学習で使われる特別なデータの形式のことです。これは、テキストが持っている意味を密度の高い情報として表す方法です。各 Embeddings は浮動小数点数のベクトルであり、ベクトル空間内の 2 つの埋め込み間の距離は、元の形式の 2 つの入力間の意味的な類似性と相関します。このベクトルを使って、2 つのテキストがどれくらい似ているかを計算できます。

## 事前準備

この Python サンプルを実行するには、以下が必要です：

- Azure OpenAI Service にアクセスできる[承認済み](https://aka.ms/oai/access) Azure サブスクリプション
- Azure OpenAI Service への Embeddings モデルのデプロイメント。
- Azure OpenAI Service の接続とモデル情報
  - OpenAI API キー
  - OpenAI Embeddings モデルのデプロイメント名。デフォルトで `text-embedding-ada-002` を使用します。
  - OpenAI API バージョン
- Python (この手順はバージョン 3.10.x でテストされています)

これらのデモには、Visual Studio Code と [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) を使用できます。


## パッケージのインストール


In [ ]:
!pip install openai

In [ ]:
import openai
openai.__version__

## 必要なライブラリと環境変数のインポート


## Azure OpenAI の設定
接続情報はセキュリティ面から直接記述するよりも、環境変数や [dotenv](https://pypi.org/project/python-dotenv/) からロードする方法をおすすめします。

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

#os.environ["AZURE_OPENAI_API_KEY"] = "Your OpenAI API Key"
#os.environ["AZURE_OPENAI_ENDPOINT"] = "https://<Your OpenAI Service>.openai.azure.com/"

#これは、モデルをデプロイしたときにデプロイメントに選んだカスタム名に対応します。
AZURE_OPENAI_DEPLOYMENT_NAME = "text-embedding-ada-002"

## ベクトル間の距離を計算する
2 つのベクトルサンプル間のコサイン類似度を計算します。

In [ ]:
# 必要なライブラリをインストールします。
#!pip install numpy matplotlib scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def calc_cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    cos = dot_product / (norm_v1 * norm_v2)
    return cos

v1 = np.array([1, 0])
v2 = np.array([1, 1])

cosine_similarity = calc_cosine_similarity(v1, v2)

print("コサイン類似度:", cosine_similarity)
print("コサイン距離:", 1 - cosine_similarity)

In [ ]:
def plot_vectors(vectors, labels):
    """
    指定されたベクトルを2次元グラフにプロットし、ラベルを表示する関数。
    
    Parameters:
    vectors (list of numpy.ndarray): プロットするベクトルのリスト。
    labels (list of str): ベクトルに対応するラベルのリスト。
    """
    # グラフの設定
    plt.figure()
    colors = ['b', 'r', 'g', 'c', 'm', 'y']  # ベクトルごとに異なる色を使用
    for i, vector in enumerate(vectors):
        plt.quiver(0, 0, vector[0], vector[1], angles='xy', scale_units='xy', scale=1, color=colors[i], label=labels[i])
        plt.text(vector[0] + 0.1, vector[1] + 0.1, f'{labels[i]} ({vector[0]}, {vector[1]})', color=colors[i], fontsize=12)

    # グラフの設定（正方形に調整）
    plt.axis("square")

    # グラフの設定（ラベル、軸の範囲など）
    plt.xlim(-2, 2)
    plt.ylim(-2, 2)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.axhline(0, color='black',linewidth=0.5)
    plt.axvline(0, color='black',linewidth=0.5)
    plt.grid(color = 'gray', linestyle = '--', linewidth = 0.5)
    plt.legend()

    # グラフの表示
    plt.show()

In [ ]:
# ベクトルの定義
v1 = np.array([1, 0])
v2 = np.array([1, 1])

# ベクトルのラベル
labels = ['v1', 'v2']

# 関数を呼び出してプロット
plot_vectors([v1, v2], labels)

## Embeddings API の使用

テキストを Embeddings モデルに渡すと、テキストは一連の浮動小数点数に変換されます。これらの Embeddings の値は、ローカルに保存するか、Azure のデータストアに保存してベクトル検索をスケールできます。

In [ ]:
from openai import AzureOpenAI

client = AzureOpenAI(
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),
  api_version="2024-02-01",
  azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

def generate_embeddings(text, model=AZURE_OPENAI_DEPLOYMENT_NAME): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding


In [ ]:
text_1 = "私はノルウェージャンフォレストキャットという猫が好きです。"
text_2 = "私はスコティッシュフォールドという猫が好きです。"

embeddings_1 = generate_embeddings(text_1)
embeddings_2 = generate_embeddings(text_2)

embeddings_1

### 2つのベクトル間のコサイン類似度を計算
コサイン類似度が 1 に近づくほど意味的に近くなります。

In [ ]:
print(calc_cosine_similarity(embeddings_1, embeddings_2))

In [ ]:
print(calc_cosine_similarity(embeddings_1, embeddings_1))

In [ ]:
text_3 = "私はトイプードルが好きです。"

embeddings_3 = generate_embeddings(text_3)
print(calc_cosine_similarity(embeddings_1, embeddings_3))

In [ ]:
text_4 = "私は Azure AI Search を使用して RAG システムを構築します。"

embeddings_4 = generate_embeddings(text_4)
print(calc_cosine_similarity(embeddings_3, embeddings_4))

## K-means を使ってクラスタを見つける
K-means の最も単純な使い方を試します。クラスタリングは、データ内の貴重な隠れたグループを発見するのに役立ちます。

In [ ]:
from sklearn.cluster import KMeans
# K-Meansクラスタリングの実行
# クラスタ数は例として3に設定します。実際のデータに応じて適宜変更してください。
kmeans = KMeans(n_clusters=3, random_state=0)
data = np.array([embeddings_1, embeddings_2, embeddings_3, embeddings_4])
clusters = kmeans.fit_predict(data)
clusters

### Embeddings の可視化（クラスタリング）
1,536 次元のベクトルを持つデータを 2 次元座標上に投影させる場合、次元削減が便利です。ただし、サンプル数が非常に少ない（この場合は4つ）ため、限定的な結果しか提供しないことに注意が必要です。下記リンクを参考にサンプル数を増やしてからデータの全体像を把握してください。

In [ ]:
from sklearn.decomposition import PCA

def plot_pca_2d(data, labels, clusters):
    """
    2次元のPCAを使用してデータをプロットする関数。
    
    :param data: 2次元に削減する高次元データ（numpy配列）
    :param labels: 各データポイントに対応するラベルのリスト
    :param clusters: 各データポイントのクラスタ割り当て（整数の配列）
    """
    if len(data) != len(labels):
        raise ValueError("データポイントの数とラベルの数が一致しません。")

    # PCAで2次元に削減
    pca = PCA(n_components=2)
    reduced_data = pca.fit_transform(data)

    # 結果の可視化（クラスタに基づく色分け）
    plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=clusters)
    for i, (x, y) in enumerate(reduced_data):
        plt.text(x, y, labels[i], fontsize=9)

    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('2D PCA Visualization with Clusters')
    plt.show()

In [ ]:
data = np.array([embeddings_1, embeddings_2, embeddings_3, embeddings_4])
labels = ['Norwegian Forest', 'Scottish Fold', 'Poodle', 'Azure AI Search']
plot_pca_2d(data, labels, clusters)

大量のデータセットを用いた Embeddings 変換やベクトル検索は [Docs](https://learn.microsoft.com/azure/ai-services/openai/tutorials/embeddings?tabs=python-new%2Ccommand-line&pivots=programming-language-python) や [00_DataIngest_AzureAISearch_PythonSDK.ipynb](../00_DataIngest_AzureAISearch_PythonSDK.ipynb) を参照してください。